In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import random

from urllib.parse import urlsplit
from collections import defaultdict

### Sample urls by hostname (merge carta and eot together)

In [22]:
data = json.load(open('data/eot_all_good.json', 'r'))
host_urls = defaultdict(list)
data = [d for d in data if d['broken'] == False]
for datum in data:
    host = urlsplit(datum['live_url']).netloc.split(':')[0]
    host_urls[host].append(datum['live_url'])
print(len(host_urls))
host_urls = {k: random.sample(v, 1) for k, v in host_urls.items()}
host_urls = defaultdict(list, random.sample(list(host_urls.items()), 6000))

data = json.load(open('data/carta_all_good.json', 'r'))
data = [d for d in data if d['broken'] == False]
for datum in data:
    host = urlsplit(datum['live_url']).netloc.split(':')[0]
    host_urls[host].append(datum['live_url'])
host_urls = {k: random.sample(v, 1) for k, v in host_urls.items()}
host_urls = dict(random.sample(list(host_urls.items()), 1500))
print(len(host_urls))
sampled_urls = [{'live_url': url[0]}  for url in host_urls.values()]
json.dump(sampled_urls, open('ground_truth_urls.json', 'w+'), indent=2)

17820
1500


In [9]:
filter_list = ['twitter.com', 'facebook.com', 'instagram.com', 'flickr.com']

eot_good_all = json.load(open('data/eot_good_all.json'))
print("Before", len(eot_good_all))
eot_good_all = [e for e in eot_good_all if not any([f in e['live_url'] for f in filter_list])]
print("After", len(eot_good_all))
eot_300 = random.sample(eot_good_all, 500)
json.dump(eot_300, open('ground-truth/ground_truth_eot_500.json', 'w'), indent=2)

Before 4093
After 3786


### Merge determinism results

In [3]:
num = 16
if os.path.exists('ground-truth/determinism_results/determinism_results.json'):
    results = {d['url']: d for d in json.load(open(f'ground-truth/determinism_results/determinism_results.json'))}
else:
    results = {}
for i in range(num):
    data = json.load(open(f'ground-truth/determinism_results/determinism_results_{i}.json'))
    print(i, len(data))
    for d in data:
        results[d['url']] = d
print(len(results))
results = list(results.values())
json.dump(results, open('ground-truth/determinism_results/determinism_results.json', 'w'), indent=2)

0 25
1 25
2 25
3 25
4 25
5 25
6 25
7 25
8 25
9 25
10 25
11 25
12 25
13 25
14 25
15 24
397


### Merge record replay results

In [5]:
num = 16
results = json.load(open(f'ground-truth/metadata/gt_tranco_metadata.json'))
for i in range(num):
    data = json.load(open(f'ground-truth/metadata/gt_tranco_metadata_{i}.json'))
    print(i, len(data))
    results.update(data)
print(len(results))
json.dump(results, open('ground-truth/metadata/gt_tranco_metadata.json', 'w'), indent=2)

0 5
1 13
2 8
3 13
4 11
5 11
6 13
7 3
8 11
9 12
10 10
11 10
12 13
13 8
14 10
15 11
162


### Construct the json for revert_rewrite

In [9]:
ground_truth = json.load(open('ground-truth/gt_diff.json'))
gt_rr = []
for obj in random.sample(ground_truth, 200):
    gt_rr.append({
        'hostname': obj['hostname'],
        'archive_url': obj['url'],
        'diff': obj['diff']
    })
json.dump(gt_rr, open('../revert_rewrite/inputs/ground_truth_200.json', 'w+'), indent=2)


### Update the json for revert_rewrite (200)

In [18]:
ground_truth_sample = json.load(open('../revert_rewrite/inputs/ground_truth_200_0.json'))
hostnames = set([g['hostname'] for g in ground_truth_sample])
gt_diff = json.load(open('ground-truth/gt_diff.json'))
gt_diff_sample = [g for g in gt_diff if g['hostname'] in hostnames]
json.dump(gt_diff_sample, open('../revert_rewrite/inputs/ground_truth_200.json', 'w+'), indent=2)

### Sample more data (merge the previous 200 in)

In [5]:
gt_sample = json.load(open('../revert_rewrite/inputs/ground_truth_200.json', 'r'))
gt_sample_map = {g['hostname']: g for g in gt_sample}
gt_all = json.load(open('ground-truth/gt_diff.json'))
gt_all_remain = [g for g in gt_all if g['hostname'] not in gt_sample_map]
gt_rr = []
for obj in gt_all_remain:
    gt_rr.append({
        'hostname': obj['hostname'],
        'archive_url': obj['url'],
        'diff': obj['diff']
    })
json.dump(gt_rr, open('../revert_rewrite/inputs/ground_truth_remain.json', 'w+'), indent=2)